# Example 5: SHIP-Classified Process Mining on FSAE Telemetry

This notebook implements the pipeline described in `overview.md`:

```
Raw Telemetry → SHIP Dispatcher → CaseGenerator → DFG / Variant Analysis
```

The SHIP Dispatcher reads a flat config list (`SHIP_EVENTS`) and classifies telemetry events
into subsystems and SHIP transition types (`error_activation` / `error_recovery`).
The CaseGenerator then builds time-windowed process mining cases around those events.

## Step 1: Imports and Data Loading

In [ ]:
import os
import sys
import warnings

import pandas as pd
import pm4py
import matplotlib.pyplot as plt
from IPython.display import display, Image

from examples.example_5.ship_config import SHIP_EVENTS
from examples.example_5.ship_dispatcher import dispatch_ship_events
from examples.example_4.case_generator import CaseGenerator
from examples.example_4.variant_visualization import visualize_chevron_variants

In [ ]:
# Reuse the parsed telemetry from example_4 — no need to re-parse the raw AIM CSV.
DATA_PATH = "../example_4/data/processed/FSAE_Endurance_Full_parsed.csv"

df = pd.read_csv(DATA_PATH)
df["time.absolute"] = pd.to_datetime(df["time.absolute"])

print(f"Loaded {len(df):,} rows x {df.shape[1]} columns")
print(f"Session: {df['time.absolute'].min()} → {df['time.absolute'].max()}")
print(f"Duration: {(df['time.absolute'].max() - df['time.absolute'].min()).total_seconds():.1f} s")

Loaded 190,587 rows × 76 columns
Session: 2016-05-14 16:27:26.010000+00:00 → 2016-05-14 16:59:11.556000+00:00
Duration: 1905.5 s


## Step 2: Data Exploration for Threshold Calibration

Review key channel statistics to confirm the thresholds in `ship_config.py` are data-appropriate.
Adjust the config and re-run the dispatcher if distributions suggest different values.

In [ ]:
channels_of_interest = {
    "Engine coolant temp (°F)": "f88.ect1_°f",
    "Oil pressure (psi)": "f88.oil.p1_psi",
    "Battery voltage (V)": "battery_v",
    "GPS satellites": "gps.nsat_#",
    "RPM": "f88.rpm_rpm",
    "Throttle (%)": "f88.tps1_%",
    "Vehicle speed (mph)": "f88.v.speed_mph",
    "FL bumpstop (unit)": "fl.bumpstop_unit",
    "Roll angle (unit)": "roll angle_unit",
}

stats = pd.DataFrame(
    {
        label: df[col].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])
        for label, col in channels_of_interest.items()
        if col in df.columns
    }
).T
display(stats.round(2))

## Step 3: SHIP Event Dispatch

The dispatcher iterates over `SHIP_EVENTS`, calls the configured `EventClassifier` method,
tags results with `subsystem` and `transition_type`, and concatenates into a single event log.

In [ ]:
with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    event_log = dispatch_ship_events(df, SHIP_EVENTS)

if w:
    print("Dispatcher warnings:")
    for warning in w:
        print(f"  {warning.message}")

print(f"\nTotal SHIP events detected: {len(event_log):,}")
display(event_log.head(10))

In [ ]:
# Event count by subsystem and transition type
summary = (
    event_log.groupby(["subsystem", "transition_type", "activity"])
    .size()
    .reset_index(name="count")
    .sort_values(["subsystem", "transition_type", "count"], ascending=[True, True, False])
)
display(summary)

In [ ]:
# Timeline: events over session time
fig, ax = plt.subplots(figsize=(14, 4))

subsystems = event_log["subsystem"].unique()
colors = plt.cm.tab10.colors
color_map = {s: colors[i % len(colors)] for i, s in enumerate(sorted(subsystems))}

for subsystem, group in event_log.groupby("subsystem"):
    ax.scatter(
        group["timestamp"],
        [subsystem] * len(group),
        c=color_map[subsystem],
        s=20,
        alpha=0.6,
        label=subsystem,
    )

ax.set_xlabel("Session time")
ax.set_title("SHIP events across the endurance session")
ax.legend(loc="upper right", fontsize=8)
plt.tight_layout()

os.makedirs("data/processed", exist_ok=True)
fig.savefig("data/processed/ship_events_timeline.png", bbox_inches="tight", dpi=150)
plt.show()

## Step 4: Case Generation

Use `CaseGenerator` from example_4 to build time-windowed cases around `error_activation` events.
Each case captures the 10 s before and 30 s after the triggering error, including any co-occurring
events from other subsystems within that window.

In [ ]:
# Use error_activation events as case triggers
activation_activities = (
    event_log[event_log["transition_type"] == "error_activation"]["activity"]
    .unique()
    .tolist()
)
print(f"Trigger activities ({len(activation_activities)}):")
for a in sorted(activation_activities):
    print(f"  {a}")

In [ ]:
cg = CaseGenerator(event_log)
cases = cg.generate_cases_time_window(
    trigger_event=activation_activities,
    time_before=10,
    time_after=30,
    case_prefix="SHIP",
)

print(f"\nCases shape: {cases.shape}")
display(cases.head(10))

In [ ]:
# Distribution of events per case
events_per_case = cases.groupby("case_id").size()
fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(events_per_case, bins=30, edgecolor="white")
ax.set_xlabel("Events per case")
ax.set_ylabel("Case count")
ax.set_title("Distribution of events per SHIP case")
plt.tight_layout()

fig.savefig("data/processed/ship_events_per_case.png", bbox_inches="tight", dpi=150)
plt.show()

## Step 5: PM4Py Event Log Preparation

Rename columns to the PM4Py standard and map `subsystem` → `org:resource`
to enable SNA metrics in the stretch goal.

In [ ]:
pm_df = cases.rename(
    columns={
        "case_id": "case:concept:name",
        "activity": "concept:name",
        "timestamp": "time:timestamp",
        "subsystem": "org:resource",
    }
).copy()

pm_df["time:timestamp"] = pd.to_datetime(pm_df["time:timestamp"])

event_log_pm4py = pm4py.format_dataframe(
    pm_df,
    case_id="case:concept:name",
    activity_key="concept:name",
    timestamp_key="time:timestamp",
)

print(f"PM4Py event log: {len(event_log_pm4py):,} events across "
      f"{event_log_pm4py['case:concept:name'].nunique()} cases")

## Step 6: DFG Analysis

Directly-Follows Graph (DFG) shows how SHIP events sequence across cases — which events
tend to follow other events within the time window.

In [ ]:
dfg, start_activities, end_activities = pm4py.discover_dfg(event_log_pm4py)

print(f"DFG: {len(dfg)} edges, {len(start_activities)} start activities, "
      f"{len(end_activities)} end activities")

# Top 15 directly-follows edges
top_edges = sorted(dfg.items(), key=lambda x: -x[1])[:15]
print("\nTop directly-follows edges:")
for (src, tgt), count in top_edges:
    print(f"  {count:4d}  {src}  →  {tgt}")

In [ ]:
os.makedirs("data/processed", exist_ok=True)

pm4py.save_vis_dfg(
    dfg,
    start_activities,
    end_activities,
    file_path="data/processed/ship_dfg.png",
)
display(Image("data/processed/ship_dfg.png"))

In [ ]:
# Performance DFG (mean time between events)
pm4py.save_vis_performance_dfg(
    *pm4py.discover_performance_dfg(event_log_pm4py),
    file_path="data/processed/ship_dfg_performance.png",
)
display(Image("data/processed/ship_dfg_performance.png"))

## Step 7: Variant Analysis

Identify unique event sequences (variants) across cases.  Each variant is a distinct
ordering of SHIP events — variants that appear repeatedly represent recurring failure
propagation patterns.

In [ ]:
variants = pm4py.get_variants(event_log_pm4py)

print(f"Total cases:    {event_log_pm4py['case:concept:name'].nunique()}")
print(f"Unique variants: {len(variants)}")
print()

# Sort by frequency
sorted_variants = sorted(variants.items(), key=lambda x: -len(x[1]))

print("Top 10 variants by case count:")
for i, (variant, case_set) in enumerate(sorted_variants[:10], 1):
    seq = " → ".join(variant)
    print(f"  [{len(case_set):3d} cases]  {seq[:120]}{'...' if len(seq) > 120 else ''}")

In [ ]:
# Chevron-style variant visualisation (reused from example_4)
top_n = 12
top_variants_seq = [list(v) for v, _ in sorted_variants[:top_n]]
top_variants_counts = [len(cases_set) for _, cases_set in sorted_variants[:top_n]]

fig = visualize_chevron_variants(
    top_variants_seq,
    top_variants_counts,
    title=f"Top {top_n} SHIP Event Variants",
)
fig.savefig("data/processed/ship_variants.png", bbox_inches="tight", dpi=150)
plt.show()

## Step 8: Subsystem-Filtered DFG

Filter by transition type to answer targeted safety questions:
- Which events precede `error_activation` events?
- What is the typical recovery sequence after an error?

In [ ]:
# DFG for error_activation events only
activation_log = event_log_pm4py[
    event_log_pm4py["concept:name"].isin(
        event_log[event_log["transition_type"] == "error_activation"]["activity"].unique()
    )
]

if len(activation_log) > 0:
    dfg_act, start_act, end_act = pm4py.discover_dfg(activation_log)
    pm4py.save_vis_dfg(
        dfg_act, start_act, end_act,
        file_path="data/processed/ship_dfg_activations.png",
    )
    print(f"Activation-only DFG: {len(dfg_act)} edges")
    display(Image("data/processed/ship_dfg_activations.png"))
else:
    print("No error_activation events in cases — adjust time window or thresholds.")

## Appendix: Stretch Goal — WaveletDenoiser

The `WaveletDenoiser` class is implemented in `wavelet_denoiser.py`.  When `pywt` is
installed (`pip install PyWavelets`), you can:

1. Denoise noisy channels before feeding them to the dispatcher.
2. Detect structural change points via energy thresholding (replacing ruptures).
3. Register energy change events in `SHIP_EVENTS` with `method: "detect_energy_change_points"`.

Example (un-comment and run after installing pywt):

In [ ]:
# from examples.example_5.wavelet_denoiser import WaveletDenoiser
#
# wd = WaveletDenoiser(df, wavelet="db4", level=4)
#
# # Denoise selected channels and rebuild the input DataFrame
# noisy_channels = ["f88.ect1_°f", "f88.oil.p1_psi", "battery_v"]
# denoised = wd.denoise_all(noisy_channels)
# df_denoised = df.copy()
# df_denoised[noisy_channels] = denoised
#
# # Add an energy-change entry to SHIP_EVENTS (stretch goal only)
# SHIP_EVENTS_WITH_ENERGY = SHIP_EVENTS + [
#     {
#         "subsystem": "Engine",
#         "transition_type": "error_activation",
#         "method": "detect_energy_change_points",
#         "args": {
#             "column": "f88.ect1_°f",
#             "event_name": "engine_temp_energy_change",
#             "threshold_sigma": 3.0,
#         },
#     },
# ]
#
# # Pass the WaveletDenoiser to the dispatcher — it routes energy-change calls to wd.
# event_log_denoised = dispatch_ship_events(df_denoised, SHIP_EVENTS_WITH_ENERGY, wd=wd)